## Goal here is to pre-process my abstracts text for topic modeling ##

In [1]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import urllib
import time
import feedparser
%matplotlib inline

from gensim import corpora, models, similarities
from gensim.matutils import Sparse2Corpus
from time import time


In [2]:
# Gonna just import the whole batch for now, b/c it's small enough and streaming isn't necessary
abstracts = pd.read_csv('./new_hope_data/arxiv_csML.csv')['summary']

In [4]:
abstracts[0]

'We discuss algorithms for estimating the Shannon entropy h of finite symbol sequences with long range correlations. In particular, we consider algorithms which estimate h from the code lengths produced by some compression algorithm. Our interest is in describing their convergence with sequence length, assuming no limits for the space and time complexities of the compression algorithms. A scaling law is proposed for extrapolation from finite sample lengths. This is applied to sequences of dynamical systems in non-trivial chaotic regimes, a 1-D cellular automaton, and to written English texts.'